In [ ]:

from dotenv import load_dotenv

load_dotenv()



In [ ]:
import requests

url = "https://storage.googleapis.com/benchmarks-artifacts/chinook/Chinook.db"

response = requests.get(url)

if response.status_code == 200:
    with open("music_store.db", "wb") as file:
        file.write(response.content)
    print("File downloaded and saved as music_store.db")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

In [ ]:
import sqlite3

conn = sqlite3.connect("music_store.db")
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Tables in the database:")
for table in tables:
    print(table[0])

conn.close()

In [ ]:
import sqlite3

import psycopg2

import os
sqlite_conn = sqlite3.connect("music_store.db")
sqlite_cursor = sqlite_conn.cursor()

pg_conn = psycopg2.connect(
    dbname= os.environ["DB_NAME"],
    user= os.environ["DB_USER"],
    password= os.environ["DB_PASSWORD"],
    host= os.environ["DB_HOST"],
    port= os.environ["DB_PORT"]
)
pg_cursor = pg_conn.cursor()

sqlite_cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = sqlite_cursor.fetchall()

for table in tables:
    table_name = table[0]
    sqlite_cursor.execute(f"PRAGMA table_info({table_name});")
    columns = sqlite_cursor.fetchall()
    column_names = [column[1] for column in columns]
    
    create_table_query = f"CREATE TABLE {table_name} ("
    create_table_query += ", ".join([f"{column[1]} {column[2].replace('NVARCHAR', 'VARCHAR').replace('DATETIME', 'TIMESTAMP')}" for column in columns])
    create_table_query += ");"
    pg_cursor.execute(create_table_query)
    
    sqlite_cursor.execute(f"SELECT * FROM {table_name};")
    rows = sqlite_cursor.fetchall()
    for row in rows:
        placeholders = ", ".join(["%s"] * len(row))
        insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({placeholders});"
        pg_cursor.execute(insert_query, row)

pg_conn.commit()
pg_cursor.close()
pg_conn.close()
sqlite_cursor.close()
sqlite_conn.close()